# Using Google Auto ML
This is a document that outlines how to get started on creating a custom object detector using Google's AutoML API (https://cloud.google.com/automl).


## Requirements
- A Google Cloud account in the Free Tier. See https://cloud.google.com/free/docs/gcp-free-tier
- Billing Information (Google claims it will not auto charge to card if doing a free trial. The free trial lasts until $300 credit is used up or until a year has past since sign up).
- Python. Supported versions are 3.5 to 3.7, and 2.7.9 or higher. Recommend >= 3.6 
- Linux, macOS, or Windows device 
Cloud SDK requires Python. Some tools bundled with Cloud SDK have additional requirements

## Setting Up Your Data (Photo Example)
The Google AutoML API accepts photos and screenshots from videos to train your custom object detector. If you are starting with unlabeled data, the first step is to upload the photos you plan to use to train your object detector onto the Google Cloud Storage https://console.cloud.google.com/storage

### Notes
- It is recommened to have 100 bounding box examples for good accuracy. >1000 photos is better.

After the photos have been uploaded, the next step is to create a CSV file of Google Storage filepaths that you plan to use to train the custom object detector. This can be done manually or with python code.